In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

data = pd.read_csv("data/climate-fever-with-emb.csv", encoding="utf-8", header=None)
embeddingsPanda = data.drop(columns=[0, 1])
embeddings = torch.Tensor(embeddingsPanda.to_numpy())
sentences = data[0]
id_claim = data[1]

C:\Users\Ange\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\Ange\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
input = "polar bear are disappearing"
inputEmb = model.encode(input)

In [12]:
def relatedSentence(threshold, inputEmb, dataEmb):
    similarityScore = util.cos_sim(inputEmb, dataEmb)
    return torch.nonzero(abs(similarityScore)> threshold).reshape(-1)

def topkRelatedSentence(k, inputEmb, dataEmb):
    similarityScore = util.cos_sim(inputEmb, dataEmb)
    return torch.topk(similarityScore, 5)[1].reshape(-1)

In [15]:
indexs = topkRelatedSentence(5, inputEmb, embeddings)
sentences.iloc[indexs].tolist()

['The polar bear population has been growing.',
 'Polar bear numbers are increasing.',
 'Global warming is driving polar bears toward extinction',
 'Polar bear numbers are increasing',
 'Polar bears are in danger of extinction as well as many other species.']

In [16]:
indexs = relatedSentence(0.4, inputEmb, embeddings)
sentences.iloc[indexs].tolist()
indexs

tensor([   0,    0,    0,    2,    0,  232,    0,  299,    0,  300,    0,  301,
           0,  302,    0,  303,    0,  304,    0,  456,    0,  506,    0,  641,
           0,  644,    0,  730,    0,  801,    0, 1014,    0, 1030,    0, 1157,
           0, 1175])